In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment2/'
FOLDERNAME = 'cs231n_final/braTS-2020-master/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/Shareddrives/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [ ]:
DATASET_PATH = '/content/drive/Shareddrives/cs231n_final/cs231n_baseline/BraTS2020'
TRAIN_PATH = f'{DATASET_PATH}/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'
VALIDATION_PATH = f'{DATASET_PATH}/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData'

# clean ds
CLEAN_DS_PATH = '/content/drive/Shareddrives/cs231n_final/cs231n_baseline/BraTS2020/3c_all_split'

# train
CLEAN_TRAIN_PATH = f'{CLEAN_DS_PATH}/train'
CLEAN_TRAIN_IMG_PATH = f'{CLEAN_TRAIN_PATH}/images'
CLEAN_TRAIN_MSK_PATH = f'{CLEAN_TRAIN_PATH}/masks'

# val
CLEAN_VAL_PATH = f'{CLEAN_DS_PATH}/val'
CLEAN_VAL_IMG_PATH = f'{CLEAN_VAL_PATH}/images'
CLEAN_VAL_MSK_PATH = f'{CLEAN_VAL_PATH}/masks'


In [ ]:
from glob import glob

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
class SimpleLogger:

    def __init__(self, debug=True):
        self.debug = debug

    def enable_debug(self):
        self.debug = True

    def disable_debug(self):
        self.debug = False

    def log(self, message, condition=True):
        if self.debug and condition:
            print(message)


logger = SimpleLogger(debug=True)


def to_categorical(y, n_classes):
    return np.eye(n_classes, dtype="uint8")[y]


class BraTSDataset(Dataset):

    def log(self, message):
        logger.log(message, condition=self.debug)

    def __init__(self, images_path, masks_path, transform=None, one_hot_target=True, debug=True, original_mask=False):
        self.images = sorted(glob(f"{images_path}/*.npy"))
        self.masks = sorted(glob(f"{masks_path}/*.npy"))
        self.transform = transform
        self.one_hot_target = one_hot_target
        self.debug = debug
        self.original_masks = original_mask
        self.log(f"images: {len(self.images)}, masks: {len(self.masks)} ")
        assert len(self.images) == len(self.masks), "images and masks lengths are not the same!"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()

        image = np.load(self.images[idx])
        mask = np.load(self.masks[idx])
        # resizing image and mask, experimental
        image = image[::2, ::2, ::2]
        if not self.original_masks:
            mask = mask[::2, ::2, ::2]
        if self.one_hot_target:
            mask = to_categorical(mask, 4)
            mask = mask[::, ::, ::, 1::]  # discard background

        image = torch.from_numpy(image).float()  # .double()
        mask = torch.from_numpy(mask)  # .float() #.long()

        return image.permute((3, 0, 1, 2)), mask.permute((3, 0, 1, 2))


def get_train_ds():
    return BraTSDataset(CLEAN_TRAIN_IMG_PATH, CLEAN_TRAIN_MSK_PATH)


def get_val_ds(full_masks=False):
    return BraTSDataset(CLEAN_VAL_IMG_PATH, CLEAN_VAL_MSK_PATH, original_mask=full_masks)


def get_dl(dataset, batch_size=32, pm=True, nw=4):
    return DataLoader(dataset, batch_size, shuffle=True, pin_memory=pm, num_workers=nw, )


In [ ]:
import nibabel as nib
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


def to_categorical(y, n_classes):
    return np.eye(n_classes, dtype="uint8")[y]


def nib_load(path):
    return nib.load(path).get_data()

In [ ]:
test_image_flair=nib.load(TRAIN_PATH + '/BraTS20_Training_355/BraTS20_Training_355_flair.nii').get_fdata()
print(test_image_flair.shape)
print(test_image_flair.min())
print(test_image_flair.max())
# #Scalers are applied to 1D so let us reshape and then reshape back to original shape.
test_image_flair=scaler.fit_transform(test_image_flair.reshape(-1, test_image_flair.shape[-1])).reshape(test_image_flair.shape)
#
#
test_image_t1=nib.load(TRAIN_PATH + '/BraTS20_Training_355/BraTS20_Training_355_t1.nii').get_fdata()
test_image_t1=scaler.fit_transform(test_image_t1.reshape(-1, test_image_t1.shape[-1])).reshape(test_image_t1.shape)
print(test_image_t1.shape)
print(test_image_t1.min())
print(test_image_t1.max())
test_image_t1ce=nib.load(TRAIN_PATH + '/BraTS20_Training_355/BraTS20_Training_355_t1ce.nii').get_fdata()
test_image_t1ce=scaler.fit_transform(test_image_t1ce.reshape(-1, test_image_t1ce.shape[-1])).reshape(test_image_t1ce.shape)
print(test_image_t1ce.shape)
print(test_image_t1ce.min())
print(test_image_t1ce.max())
test_image_t2=nib.load(TRAIN_PATH + '/BraTS20_Training_355/BraTS20_Training_355_t2.nii').get_fdata()
test_image_t2=scaler.fit_transform(test_image_t2.reshape(-1, test_image_t2.shape[-1])).reshape(test_image_t2.shape)
print(test_image_t2.shape)
print(test_image_t2.min())
print(test_image_t2.max())
#
test_mask=nib.load(TRAIN_PATH + '/BraTS20_Training_355/BraTS20_Training_355_seg.nii').get_fdata()
test_mask=test_mask.astype(np.uint8)
print(test_mask.shape)
print(test_mask.min())
print(test_mask.max())

(240, 240, 155)
0.0
1854.603271484375
(240, 240, 155)
0.0
1.0
(240, 240, 155)
0.0
1.0
(240, 240, 155)
0.0
1.0
(240, 240, 155)
0
4


In [ ]:
print(np.unique(test_mask))
test_mask[test_mask==4] = 3
print(np.unique(test_mask))

[0 1 2 4]
[0 1 2 3]


In [ ]:
import glob

In [ ]:
t2_list = sorted(glob.glob(f'{TRAIN_PATH}/*/*t2.nii'))  # 369 training instances
t1ce_list = sorted(glob.glob(f'{TRAIN_PATH}/*/*t1ce.nii'))
flair_list = sorted(glob.glob(f'{TRAIN_PATH}/*/*flair.nii'))
mask_list = sorted(glob.glob(f'{TRAIN_PATH}/*/*seg.nii'))

In [ ]:
v_t2_list = sorted(glob.glob(f'{VALIDATION_PATH}/*/*t2.nii'))  # 125 validation data instances
v_t1ce_list = sorted(glob.glob(f'{VALIDATION_PATH}/*/*t1ce.nii'))
v_flair_list = sorted(glob.glob(f'{VALIDATION_PATH}/*/*flair.nii'))
v_mask_list = sorted(glob.glob(f'{VALIDATION_PATH}/*/*seg.nii'))

In [ ]:
print(len(v_flair_list))

125


In [ ]:
for img in range(len(t2_list)):
    print("image index at:", img)

    temp_image_t2 = nib.load(t2_list[img]).get_fdata()
    temp_image_t2 = scaler.fit_transform(temp_image_t2.reshape(-1, temp_image_t2.shape[-1])).reshape(
        temp_image_t2.shape)

    temp_image_t1ce = nib.load(t1ce_list[img]).get_fdata()
    temp_image_t1ce = scaler.fit_transform(temp_image_t1ce.reshape(-1, temp_image_t1ce.shape[-1])).reshape(
        temp_image_t1ce.shape)

    temp_image_flair = nib.load(flair_list[img]).get_fdata()
    temp_image_flair = scaler.fit_transform(temp_image_flair.reshape(-1, temp_image_flair.shape[-1])).reshape(
        temp_image_flair.shape)

    temp_mask = nib.load(mask_list[img]).get_fdata()
    temp_mask = temp_mask.astype(np.uint8)
    temp_mask[temp_mask == 4] = 3

    temp_combined_images = np.stack([temp_image_flair, temp_image_t1ce, temp_image_t2], axis=3)

    temp_combined_images = temp_combined_images[56:184, 56:184, 13:141]
    temp_mask = temp_mask[56:184, 56:184, 13:141]

    val, counts = np.unique(temp_mask, return_counts=True)

    # temp_mask= to_categorical(temp_mask, num_classes=4)
    np.save(f'{TRAIN_PATH}/3c_all/images/image_' + str(img) + '.npy', temp_combined_images)
    np.save(f'{TRAIN_PATH}/3c_all/masks/mask_' + str(img) + '.npy', temp_mask)

image index at: 0
image index at: 1
image index at: 2
image index at: 3
image index at: 4
image index at: 5
image index at: 6
image index at: 7
image index at: 8
image index at: 9
image index at: 10
image index at: 11
image index at: 12
image index at: 13
image index at: 14
image index at: 15
image index at: 16
image index at: 17
image index at: 18
image index at: 19
image index at: 20
image index at: 21
image index at: 22
image index at: 23
image index at: 24
image index at: 25
image index at: 26
image index at: 27
image index at: 28
image index at: 29
image index at: 30
image index at: 31
image index at: 32
image index at: 33
image index at: 34
image index at: 35
image index at: 36
image index at: 37
image index at: 38
image index at: 39
image index at: 40
image index at: 41
image index at: 42
image index at: 43
image index at: 44
image index at: 45
image index at: 46
image index at: 47
image index at: 48
image index at: 49
image index at: 50
image index at: 51
image index at: 52
ima

KeyboardInterrupt: ignored

In [ ]:
!pip install split-folders[full]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders # the original data's validation cannot be used as test, due to a lack of masks

clean_data_folder = '/content/drive/Shareddrives/cs231n_final/cs231n_baseline/BraTS2020/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/3c_all'
target_folder = '/content/drive/Shareddrives/cs231n_final/cs231n_baseline/BraTS2020/3c_all_split'
splitfolders.ratio(clean_data_folder, output=target_folder, seed=42, ratio=(.6, .4), group_prefix=None)

Copying files: 4 files [00:07,  1.91s/ files]

KeyboardInterrupt: ignored

In [ ]:
from tqdm import tqdm


# must load the model to the device and also prep the loss function with model parameters before using this function
# also do not forget to initialize the optimizer with the desired learning rate
def train(model, epochs=1, training_loader=None, loss_fn=None, device=None,
          optimizer: torch.optim.Optimizer = None, from_epoch=0):  # from_epoch is for the resume mode
    for epoch in range(from_epoch, epochs):
        tq_dl = tqdm(training_loader)
        for idx, (image, mask) in enumerate(tq_dl):
            image, mask = image.to(device), mask.to(device)
            # forward pass
            out = model(image)
            loss = loss_fn(out, mask)
            # backward pass
            optimizer.zero_grad()
            loss.backward()

            # optimize
            optimizer.step()

            tq_dl.set_description(f"At epoch [{epoch + 1}/{epochs}]")
            tq_dl.set_postfix(loss=loss.item())  # acc, ...


def check_accuracy(data_loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)  # .unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                    (preds + y).sum() + 1e-8
            )

    print(
        f"Results: {num_correct}/{num_pixels} with accuracy {num_correct / num_pixels * 100:.4f}"
    )
    print(f"Dice score: {dice_score / len(data_loader)}")
    model.train()


# our labels
# 1: Necrotic and Non-enhancing tumor core (NCR/NET)
# 2: Peritumoral Edema (ED)
# 3: GD-enhancing tumor (ET)
# common stuff in other papers:
# TC -> 1 + 3
# ET -> 3
# WT -> TC + ED: all
def check_accuracy_v2(data_loader, model, device="cuda"):
    num_correct = {'TC': 0, 'ET': 0, 'WT': 0}
    num_pixels = {'TC': 0, 'ET': 0, 'WT': 0}
    dice_score = {'TC': 0, 'ET': 0, 'WT': 0}
    model.eval()

    with torch.no_grad():
        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)  # .unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds >= 0.5).float()
            TC_pred = (preds[:, 0] + preds[:, 2] >= 1).float()
            TC_real = (y[:, 0] + y[:, 2] >= 1).float()
            ET_pred = preds[:, 2]
            ET_real = y[:, 2]
            WT_pred = (preds[:, 0] + preds[:, 1] + preds[:, 2] >= 1).float()
            WT_real = (y[:, 0] + y[:, 1] + y[:, 2] >= 1).float()

            num_correct['TC'] += (TC_pred == TC_real).sum()
            num_pixels['TC'] += torch.numel(TC_pred)
            dice_score['TC'] += (2 * (TC_pred * TC_real).sum()) / (
                    (TC_pred + TC_real).sum() + 1e-8)

            num_correct['ET'] += (ET_pred == ET_real).sum()
            num_pixels['ET'] += torch.numel(ET_pred)
            dice_score['ET'] += (2 * (ET_pred * ET_real).sum()) / (
                    (ET_pred + ET_real).sum() + 1e-8)

            num_correct['WT'] += (WT_pred == WT_real).sum()
            num_pixels['WT'] += torch.numel(WT_pred)
            dice_score['WT'] += (2 * (WT_pred * WT_real).sum()) / (
                    (WT_pred + WT_real).sum() + 1e-8)

    # print(
    #     f"Results (TC,ET,WT): ({num_correct['TC']}/{num_pixels['TC']}) with accuracy {num_correct / num_pixels * 100:.4f}"
    # )

    print(
        f" Accuracy (TC,ET,WT): \n --> {num_correct['TC'] / num_pixels['TC'] * 100:.4f} , {num_correct['ET'] / num_pixels['ET'] * 100:.4f}, {num_correct['WT'] / num_pixels['WT'] * 100:.4f}")
    print(
        f"Dice Score (TC,ET,WT): \n {dice_score['TC'] / len(data_loader)} , {dice_score['ET'] / len(data_loader)}, {dice_score['WT'] / len(data_loader)}")
    model.train()

def check_accuracy_v3(data_loader, model, device="cuda"):  # requires full masks
    num_correct = {'TC': 0, 'ET': 0, 'WT': 0}
    num_pixels = {'TC': 0, 'ET': 0, 'WT': 0}
    dice_score = {'TC': 0, 'ET': 0, 'WT': 0}
    model.eval()

    with torch.no_grad():
        for x, y1 in data_loader:
            y = y1[:, :, ::2, ::2, ::2]  # get the halved mask

            x = x.to(device)
            y = y.to(device)  # .unsqueeze(1)
            y, y1 = y1, y
            original_preds = torch.sigmoid(model(x))

            preds = simple_trilinear_interpolation(original_preds)  # here or next?

            preds = (preds >= 0.5).float()
            TC_pred = (preds[:, 0] + preds[:, 2] >= 1).float()
            TC_real = (y[:, 0] + y[:, 2] >= 1).float()
            ET_pred = preds[:, 2]
            ET_real = y[:, 2]
            WT_pred = (preds[:, 0] + preds[:, 1] + preds[:, 2] >= 1).float()
            WT_real = (y[:, 0] + y[:, 1] + y[:, 2] >= 1).float()

            num_correct['TC'] += (TC_pred == TC_real).sum()
            num_pixels['TC'] += torch.numel(TC_pred)
            dice_score['TC'] += (2 * (TC_pred * TC_real).sum()) / (
                    (TC_pred + TC_real).sum() + 1e-8)

            num_correct['ET'] += (ET_pred == ET_real).sum()
            num_pixels['ET'] += torch.numel(ET_pred)
            dice_score['ET'] += (2 * (ET_pred * ET_real).sum()) / (
                    (ET_pred + ET_real).sum() + 1e-8)

            num_correct['WT'] += (WT_pred == WT_real).sum()
            num_pixels['WT'] += torch.numel(WT_pred)
            dice_score['WT'] += (2 * (WT_pred * WT_real).sum()) / (
                    (WT_pred + WT_real).sum() + 1e-8)

    # print(
    #     f"Results (TC,ET,WT): ({num_correct['TC']}/{num_pixels['TC']}) with accuracy {num_correct / num_pixels * 100:.4f}"
    # )

    print(
        f" Accuracy (TC,ET,WT): \n --> {num_correct['TC'] / num_pixels['TC'] * 100:.4f} , {num_correct['ET'] / num_pixels['ET'] * 100:.4f}, {num_correct['WT'] / num_pixels['WT'] * 100:.4f}")
    print(
        f"Dice Score (TC,ET,WT): \n {dice_score['TC'] / len(data_loader)} , {dice_score['ET'] / len(data_loader)}, {dice_score['WT'] / len(data_loader)}")
    model.train()

def get_all_metrics(data_loader, model, device="cuda"):  # requires full masks
    num_correct = {'TC': 0, 'ET': 0, 'WT': 0}
    num_true_predicts = {'TC': 0, 'ET': 0, 'WT': 0}
    num_true_labels = {'TC': 0, 'ET': 0, 'WT': 0}
    num_pixels = {'TC': 0, 'ET': 0, 'WT': 0}
    dice_score = {'TC': 0, 'ET': 0, 'WT': 0}
    iou_score = {'TC': 0, 'ET': 0, 'WT': 0}

    EPS = 1e-16
    model.eval()

    with torch.no_grad():
        for x, y1 in data_loader:
            y = y1[:, :, ::2, ::2, ::2]  # get the halved mask

            x = x.to(device)
            y = y.to(device)  # .unsqueeze(1)
            y, y1 = y1, y
            original_preds = torch.sigmoid(model(x))

            preds = simple_trilinear_interpolation(original_preds)  # here or next?

            preds = (preds >= 0.5).float()
            TC_pred = (preds[:, 0] + preds[:, 2] > 0.9).float()
            TC_real = (y[:, 0] + y[:, 2] > 0.9).float()
            ET_pred = preds[:, 2]
            ET_real = y[:, 2]
            WT_pred = (preds[:, 0] + preds[:, 1] + preds[:, 2] > 0.9).float()
            WT_real = (y[:, 0] + y[:, 1] + y[:, 2] > 0.9).float()

            num_correct['TC'] += (TC_pred == TC_real).sum()
            num_pixels['TC'] += torch.numel(TC_pred)
            num_true_predicts['TC'] += (TC_pred > .9).sum()  # those that are 1
            num_true_labels['TC'] += (TC_real > .9).sum()
            dice_score['TC'] += (2 * (TC_pred * TC_real).sum()) / ((TC_pred + TC_real).sum() + EPS)
            iou_score['TC'] += (TC_pred * TC_real).sum() / ((TC_pred + TC_real).sum() - (TC_pred * TC_real).sum() + EPS)

            num_correct['ET'] += (ET_pred == ET_real).sum()
            num_pixels['ET'] += torch.numel(ET_pred)
            num_true_predicts['ET'] += (ET_pred > .9).sum()
            num_true_labels['ET'] += (ET_real > .9).sum()
            dice_score['ET'] += (2 * (ET_pred * ET_real).sum()) / ((ET_pred + ET_real).sum() + EPS)
            iou_score['ET'] += (ET_pred * ET_real).sum() / ((ET_pred + ET_real).sum() - (ET_pred * ET_real).sum() + EPS)

            num_correct['WT'] += (WT_pred == WT_real).sum()
            num_pixels['WT'] += torch.numel(WT_pred)
            num_true_predicts['WT'] += (WT_pred > .9).sum()
            num_true_labels['WT'] += (WT_real > .9).sum()
            dice_score['WT'] += (2 * (WT_pred * WT_real).sum()) / ((WT_pred + WT_real).sum() + EPS)
            iou_score['WT'] += (WT_pred * WT_real).sum() / ((WT_pred + WT_real).sum() - (WT_pred * WT_real).sum() + EPS)

    # precision: correct true predicts / all true predicts -> TP / (TP + FP)

    precision = {'TC': num_correct['TC'] / num_true_predicts['TC'] * 100,
                 'ET': num_correct['ET'] / num_true_predicts['ET'] * 100,
                 'WT': num_correct['WT'] / num_true_predicts['WT'] * 100}
    # recall: correct true predicts / all true labels -> TP / (TP + FN)

    recall = {'TC': num_correct['TC'] / num_true_labels['TC'] * 100,
              'ET': num_correct['ET'] / num_true_labels['ET'] * 100,
              'WT': num_correct['WT'] / num_true_labels['WT'] * 100}

    # f1 is actually the same as Dice Coefficient, or is it?
    f1 = {'TC': 2 * precision['TC'] * recall['TC'] / (precision['TC'] + recall['TC']),
          'ET': 2 * precision['ET'] * recall['ET'] / (precision['ET'] + recall['ET']),
          'WT': 2 * precision['WT'] * recall['WT'] / (precision['WT'] + recall['WT'])}

    print(
        f" Accuracy (TC,ET,WT): \n {num_correct['TC'] / num_pixels['TC'] * 100:.4f} , {num_correct['ET'] / num_pixels['ET'] * 100:.4f}, {num_correct['WT'] / num_pixels['WT'] * 100:.4f}")
    print(
        f"Dice Score (TC,ET,WT): \n {dice_score['TC'] / len(data_loader)} , {dice_score['ET'] / len(data_loader)}, {dice_score['WT'] / len(data_loader)}")
    print(
        f"IoU Score (TC,ET,WT): \n {iou_score['TC'] / len(data_loader)} , {iou_score['ET'] / len(data_loader)}, {iou_score['WT'] / len(data_loader)}")

    print(
        f" Precision (TC,ET,WT): \n --> {precision['TC'] / len(data_loader):.4f} , {precision['ET'] / len(data_loader):.4f}, {precision['WT'] / len(data_loader):.4f}")
    print(
        f" Recall (TC,ET,WT): \n --> {recall['TC'] / len(data_loader):.4f} , {recall['ET'] / len(data_loader):.4f}, {recall['WT'] / len(data_loader):.4f}")
    print(
        f" F1-score (TC,ET,WT): \n --> {f1['TC'] / len(data_loader):.4f} , {f1['ET'] / len(data_loader):.4f}, {f1['WT'] / len(data_loader):.4f}")

    model.train()

    return None

# This one gives true average on all, and is final
def get_all_metrics_2(data_loader, model, device="cuda"):  # requires full masks
    num_correct = {'TC': 0, 'ET': 0, 'WT': 0}
    num_true_predicts = {'TC': 0, 'ET': 0, 'WT': 0}
    num_true_labels = {'TC': 0, 'ET': 0, 'WT': 0}
    num_pixels = {'TC': 0, 'ET': 0, 'WT': 0}
    dice_score = {'TC': 0, 'ET': 0, 'WT': 0}
    iou_score = {'TC': 0, 'ET': 0, 'WT': 0}

    results = {'TC': {'real': [], 'pred': []}, 'ET': {'real': [], 'pred': []}, 'WT': {'real': [], 'pred': []}}

    EPS = 1e-9
    model.eval()

    with torch.no_grad():
        for x, y1 in data_loader:
            y = y1[:, :, ::2, ::2, ::2]  # get the halved mask

            x = x.to(device)
            y = y.to(device)  # .unsqueeze(1)
            y, y1 = y1, y
            original_preds = torch.sigmoid(model(x))

            preds = simple_trilinear_interpolation(original_preds)  # here or next?

            # preds = (preds >= 0.5).float() # can't do it here, need the probabilities for auc_roc
            for i in range(preds.shape[0]):
                # TC_pred = (preds[i, 0] + preds[i, 2] > 0.9).float()
                # ET_pred = preds[i, 2]
                # WT_pred = (preds[i, 0] + preds[i, 1] + preds[i, 2] > 0.9).float()

                TC_real = (y[i, 0] + y[i, 2] > 0.9).float()
                ET_real = y[i, 2]
                WT_real = (y[i, 0] + y[i, 1] + y[i, 2] > 0.9).float()

                TC_pred = (preds[i, 0] + preds[i, 2]).float()
                ET_pred = preds[i, 2]
                WT_pred = (preds[i, 0] + preds[i, 1] + preds[i, 2]).float()

                # for auc_roc we need non-thresholded probabilities
                results['TC']['pred'].append(TC_pred.flatten().cpu().numpy())
                results['TC']['real'].append(TC_real.flatten().cpu().numpy())
                results['ET']['pred'].append(ET_pred.flatten().cpu().numpy())
                results['ET']['real'].append(ET_real.flatten().cpu().numpy())
                results['WT']['pred'].append(WT_pred.flatten().cpu().numpy())
                results['WT']['real'].append(WT_real.flatten().cpu().numpy())

                # Thresholding again
                TC_pred = (TC_pred >= 0.5).float()
                ET_pred = (ET_pred >= 0.5).float()
                WT_pred = (WT_pred >= 0.5).float()

                num_correct['TC'] += (TC_pred == TC_real).sum()
                num_pixels['TC'] += torch.numel(TC_pred)
                num_true_predicts['TC'] += (TC_pred > .9).sum()  # those that are 1
                num_true_labels['TC'] += (TC_real > .9).sum()
                dice_score['TC'] += (2 * (TC_pred * TC_real).sum()) / ((TC_pred + TC_real).sum() + EPS)
                iou_score['TC'] += (TC_pred * TC_real).sum() / (
                        (TC_pred + TC_real).sum() - (TC_pred * TC_real).sum() + EPS)

                num_correct['ET'] += (ET_pred == ET_real).sum()
                num_pixels['ET'] += torch.numel(ET_pred)
                num_true_predicts['ET'] += (ET_pred > .9).sum()
                num_true_labels['ET'] += (ET_real > .9).sum()
                dice_score['ET'] += (2 * (ET_pred * ET_real).sum()) / ((ET_pred + ET_real).sum() + EPS)
                iou_score['ET'] += (ET_pred * ET_real).sum() / (
                        (ET_pred + ET_real).sum() - (ET_pred * ET_real).sum() + EPS)

                num_correct['WT'] += (WT_pred == WT_real).sum()
                num_pixels['WT'] += torch.numel(WT_pred)
                num_true_predicts['WT'] += (WT_pred > .9).sum()
                num_true_labels['WT'] += (WT_real > .9).sum()
                dice_score['WT'] += (2 * (WT_pred * WT_real).sum()) / ((WT_pred + WT_real).sum() + EPS)
                iou_score['WT'] += (WT_pred * WT_real).sum() / (
                        (WT_pred + WT_real).sum() - (WT_pred * WT_real).sum() + EPS)

    # precision: correct true predicts / all true predicts -> TP / (TP + FP)

    precision = {'TC': num_correct['TC'] / num_true_predicts['TC'] * 100,
                 'ET': num_correct['ET'] / num_true_predicts['ET'] * 100,
                 'WT': num_correct['WT'] / num_true_predicts['WT'] * 100}
    # recall: correct true predicts / all true labels -> TP / (TP + FN)

    recall = {'TC': num_correct['TC'] / num_true_labels['TC'] * 100,
              'ET': num_correct['ET'] / num_true_labels['ET'] * 100,
              'WT': num_correct['WT'] / num_true_labels['WT'] * 100}

    # f1 is actually the same as Dice Coefficient, or is it?
    f1 = {'TC': 2 * precision['TC'] * recall['TC'] / (precision['TC'] + recall['TC']),
          'ET': 2 * precision['ET'] * recall['ET'] / (precision['ET'] + recall['ET']),
          'WT': 2 * precision['WT'] * recall['WT'] / (precision['WT'] + recall['WT'])}

    items_size = preds.shape[0] * len(data_loader)

    print(
        f" Accuracy (TC,ET,WT): \n {num_correct['TC'] / num_pixels['TC'] * 100:.4f} , {num_correct['ET'] / num_pixels['ET'] * 100:.4f}, {num_correct['WT'] / num_pixels['WT'] * 100:.4f}")
    print(
        f"Dice Score (TC,ET,WT): \n {dice_score['TC'] / items_size} , {dice_score['ET'] / items_size}, {dice_score['WT'] / items_size}")
    print(
        f"IoU Score (TC,ET,WT): \n {iou_score['TC'] / items_size} , {iou_score['ET'] / items_size}, {iou_score['WT'] / items_size}")

    print(
        f" Precision (TC,ET,WT): \n --> {precision['TC'] / items_size:.4f} , {precision['ET'] / items_size:.4f}, {precision['WT'] / items_size:.4f}")
    print(
        f" Recall (TC,ET,WT): \n --> {recall['TC'] / items_size:.4f} , {recall['ET'] / items_size:.4f}, {recall['WT'] / items_size:.4f}")
    print(
        f" F1-score (TC,ET,WT): \n --> {f1['TC'] / items_size:.4f} , {f1['ET'] / items_size:.4f}, {f1['WT'] / items_size:.4f}")

    model.train()

    return results


In [ ]:
from collections import OrderedDict
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
#Layers
def default_norm_layer(planes, groups=16):
    groups_ = min(groups, planes)
    if planes % groups_ > 0:
        divisor = 16
        while planes % divisor > 0:
            divisor /= 2
        groups_ = int(planes // divisor)
    return nn.GroupNorm(groups_, planes)


def get_norm_layer(norm_type="group"):
    if "group" in norm_type:
        try:
            grp_nb = int(norm_type.replace("group", ""))
            return lambda planes: default_norm_layer(planes, groups=grp_nb)
        except ValueError as e:
            print(e)
            print('using default group number')
            return default_norm_layer
    elif norm_type == "none":
        return None
    else:
        return lambda x: nn.InstanceNorm3d(x, affine=True)


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1, bias=False):
    """3x3 convolution with padding"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=bias, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1, bias=True):
    """1x1 convolution"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=1, stride=stride, bias=bias)


class ConvBnRelu(nn.Sequential):

    def __init__(self, inplanes, planes, norm_layer=None, dilation=1, dropout=0):
        if norm_layer is not None:
            super(ConvBnRelu, self).__init__(
                OrderedDict(
                    [
                        ('conv', conv3x3(inplanes, planes, dilation=dilation)),
                        ('bn', norm_layer(planes)),
                        ('relu', nn.ReLU(inplace=True)),
                        ('dropout', nn.Dropout(p=dropout)),
                    ]
                )
            )
        else:
            super(ConvBnRelu, self).__init__(
                OrderedDict(
                    [
                        ('conv', conv3x3(inplanes, planes, dilation=dilation, bias=True)),
                        ('relu', nn.ReLU(inplace=True)),
                        ('dropout', nn.Dropout(p=dropout)),
                    ]
                )
            )


class UBlock(nn.Sequential):
    """Unet mainstream downblock.
    """

    def __init__(self, inplanes, midplanes, outplanes, norm_layer, dilation=(1, 1), dropout=0):
        super(UBlock, self).__init__(
            OrderedDict(
                [
                    ('ConvBnRelu1', ConvBnRelu(inplanes, midplanes, norm_layer, dilation=dilation[0], dropout=dropout)),
                    (
                        'ConvBnRelu2',
                        ConvBnRelu(midplanes, outplanes, norm_layer, dilation=dilation[1], dropout=dropout)),
                ])
        )


class UBlockCbam(nn.Sequential):
    def __init__(self, inplanes, midplanes, outplanes, norm_layer, dilation=(1, 1), dropout=0):
        super(UBlockCbam, self).__init__(
            OrderedDict(
                [
                    ('UBlock', UBlock(inplanes, midplanes, outplanes, norm_layer, dilation=dilation, dropout=dropout)),
                    ('CBAM', CBAM(outplanes, norm_layer=norm_layer)),
                ])
        )


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


#if __name__ == '__main__':
    #print(UBlock(4, 4))


class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1,
                 norm_layer=None):
        super(BasicConv, self).__init__()
        bias = False
        self.out_channels = out_planes
        self.conv = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding,
                              dilation=dilation, groups=groups, bias=bias)
        #self.bn = norm_layer(out_planes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        #x = self.bn(x)
        x = self.relu(x)
        return x


class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)


class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(inplace=True),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
        )
        self.pool_types = pool_types

    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type == 'avg':
                avg_pool = F.avg_pool3d(x, (x.size(2), x.size(3), x.size(4)), stride=(x.size(2), x.size(3), x.size(4)))
                channel_att_raw = self.mlp(avg_pool)
            elif pool_type == 'max':
                max_pool = F.max_pool3d(x, (x.size(2), x.size(3), x.size(4)), stride=(x.size(2), x.size(3), x.size(4)))
                channel_att_raw = self.mlp(max_pool)
            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = torch.sigmoid(channel_att_sum).unsqueeze(2).unsqueeze(3).unsqueeze(4).expand_as(x)
        return x * scale


class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1), torch.mean(x, 1).unsqueeze(1)), dim=1)


class SpatialGate(nn.Module):
    def __init__(self, norm_layer=None):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size - 1) // 2, norm_layer=norm_layer)

    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = torch.sigmoid(x_out)  # broadcasting
        return x * scale


class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=None, norm_layer=None):
        super(CBAM, self).__init__()
        if pool_types is None:
            pool_types = ['avg', 'max']
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.SpatialGate = SpatialGate(norm_layer)

    def forward(self, x):
        x_out = self.ChannelGate(x)
        x_out = self.SpatialGate(x_out)
        return x_out

In [ ]:
class IntermediateSequential(nn.Sequential):
    def __init__(self, *args, return_intermediate=True):
        super().__init__(*args)
        self.return_intermediate = return_intermediate

    def forward(self, input):
        if not self.return_intermediate:
            return super().forward(input)

        intermediate_outputs = {}
        output = input
        for name, module in self.named_children():
            output = intermediate_outputs[name] = module(output)

        return output, intermediate_outputs

In [ ]:
#Transformer
class SelfAttention(nn.Module):
    def __init__(
        self, dim, heads=8, qkv_bias=False, qk_scale=None, dropout_rate=0.0
    ):
        super().__init__()
        self.num_heads = heads
        head_dim = dim // heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(dropout_rate)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(dropout_rate)

    def forward(self, x):
        B, N, C = x.shape
        qkv = (
            self.qkv(x)
            .reshape(B, N, 3, self.num_heads, C // self.num_heads)
            .permute(2, 0, 3, 1, 4)
        )
        q, k, v = (
            qkv[0],
            qkv[1],
            qkv[2],
        )  # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x


class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x):
        return self.fn(self.norm(x))


class PreNormDrop(nn.Module):
    def __init__(self, dim, dropout_rate, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fn = fn

    def forward(self, x):
        return self.dropout(self.fn(self.norm(x)))


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout_rate):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(p=dropout_rate),
        )

    def forward(self, x):
        return self.net(x)


class TransformerModel(nn.Module):
    def __init__(
        self,
        dim,
        depth,
        heads,
        mlp_dim,
        dropout_rate=0.1,
        attn_dropout_rate=0.1,
    ):
        super().__init__()
        layers = []
        for _ in range(depth):
            layers.extend(
                [
                    Residual(
                        PreNormDrop(
                            dim,
                            dropout_rate,
                            SelfAttention(dim, heads=heads, dropout_rate=attn_dropout_rate),
                        )
                    ),
                    Residual(
                        PreNorm(dim, FeedForward(dim, mlp_dim, dropout_rate))
                    ),
                ]
            )
            # dim = dim / 2
        self.net = IntermediateSequential(*layers)


    def forward(self, x):
        return self.net(x)

In [ ]:
#Positional Encoding
class FixedPositionalEncoding(nn.Module):
    def __init__(self, embedding_dim, max_length=512):
        super(FixedPositionalEncoding, self).__init__()

        pe = torch.zeros(max_length, embedding_dim)
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, embedding_dim, 2).float()
            * (-torch.log(torch.tensor(10000.0)) / embedding_dim)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0), :]
        return x


class LearnedPositionalEncoding(nn.Module):
    def __init__(self, max_position_embeddings, embedding_dim, seq_length):
        super(LearnedPositionalEncoding, self).__init__()

        self.position_embeddings = nn.Parameter(torch.zeros(1, 512, 256)) #8x

    def forward(self, x, position_ids=None):

        position_embeddings = self.position_embeddings
        return x + position_embeddings

class LearnedPositionalEncoding_4(nn.Module):
    def __init__(self, max_position_embeddings, embedding_dim, seq_length):
        super(LearnedPositionalEncoding_4, self).__init__()

        self.position_embeddings = nn.Parameter(torch.zeros(1, 2048, 512)) #8x

    def forward(self, x, position_ids=None):

        position_embeddings = self.position_embeddings
        return x + position_embeddings

In [ ]:
class SimpleLogger:

    def __init__(self, debug=True):
        self.debug = debug

    def enable_debug(self):
        self.debug = True

    def disable_debug(self):
        self.debug = False

    def log(self, message, condition=True):
        if self.debug and condition:
            print(message)


logger = SimpleLogger(debug=True)

"""
class DoubleConv3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # 1 + (L - l + 2P)/s
        self.conv = nn.Sequential(
            # 1 + out - 3 + 2 = out
            nn.Conv3d(in_channels, out_channels, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, 3, stride=1, padding=1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, inputs):
        return self.conv(inputs)
"""
#3D UNET
class Base3DUNet(BiTransformerUnet):
    # the default dataset has 3 channels of data ->  T1CE, T2, FLAIR
    # The output has background, NCR/NET, ED, ET
    def __init__(self, in_channels=3, out_channels=4, features=[64, 128, 256, 512], up_sample=False):
        super(BiTransformerUnet, self).__init__(
            img_dim=img_dim,
            patch_dim=patch_dim,
            num_channels=num_channels,
            embedding_dim_4=embedding_dim_4,
            embedding_dim=embedding_dim,
            num_heads=num_heads,
            num_layers=num_layers,
            hidden_dim=hidden_dim,
            dropout_rate=dropout_rate,
            attn_dropout_rate=attn_dropout_rate,
            conv_patch_representation=conv_patch_representation,
            positional_encoding_type=positional_encoding_type,
        )
        # 1 + (L - l + 2P)/s
        # 1 + (L - 2)/2 = L
        self.pooling = nn.MaxPool3d(kernel_size=2, stride=2)

        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()

        # Each Layer - number of filters , see UNet architecture
        input_channels = in_channels

        for feature in features:
            self.downs.append(BiTransformerUnet(input_channels, feature))
            input_channels = feature

        for feature in reversed(features):
            if up_sample:
                self.ups.append(nn.Upsample(scale_factor=2, mode="trilinear", align_corners=True))
                self.ups.append(BiTransformerUnet(feature * 3, feature))
                # *3 because upsample does not change the number of channels
            else:
                self.ups.append(nn.ConvTranspose3d(feature * 2, feature, kernel_size=2, stride=2))
                self.ups.append(BiTransformerUnet(feature * 2, feature))

        self.bottleneck = BiTransformerUnet(features[-1], features[-1] * 2)  # this connects downs to ups

        self.output_conv = nn.Conv3d(features[0], out_channels, kernel_size=1)  # last layer - feature compression

    def forward(self, inputs):
        skips = []

        x = inputs
        for down in self.downs:
            x = down(x)
            skips.append(x)
            x = self.pooling(x)

        x = self.bottleneck(x)

        for idx in range(0, len(self.ups), 2):  # going up 2 steps, as each step has convTranspose and DoubleConv
            x = self.ups[idx](x)  # up sampling w/ the convTranspose
            skip_connection = skips.pop()  # give me the last skip I added, to add it first on the ups
            x = torch.cat((skip_connection, x), dim=1)  # dim 0 is batch, dim 1 is the channels
            x = self.ups[idx + 1](x)  # double conv

        return self.output_conv(x)

class BiTransformerUnet(nn.Module):
    def __init__(
        self,
        img_dim,
        patch_dim,
        num_channels,
        embedding_dim_4,
        embedding_dim,
        num_heads,
        num_layers,
        hidden_dim,
        dropout_rate=0.0,
        attn_dropout_rate=0.0,
        conv_patch_representation=True,
        positional_encoding_type="learned",
    ):
        super(Attention3UNet, self).__init__()

        assert embedding_dim % num_heads == 0
        assert img_dim % patch_dim == 0

        self.img_dim = img_dim
        self.embedding_dim_4 = embedding_dim_4
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.patch_dim = patch_dim
        self.num_channels = num_channels
        self.dropout_rate = dropout_rate
        self.attn_dropout_rate = attn_dropout_rate
        self.conv_patch_representation = conv_patch_representation

        self.num_patches = int((img_dim // patch_dim) ** 3)
        self.seq_length = self.num_patches
        self.flatten_dim = 128 * num_channels

        self.linear_encoding = nn.Linear(self.flatten_dim, self.embedding_dim)
        if positional_encoding_type == "learned":
            self.position_encoding = LearnedPositionalEncoding(
                self.seq_length, self.embedding_dim, self.seq_length
            )
            self.position_encoding_4 = LearnedPositionalEncoding_4(
                self.seq_length, self.embedding_dim_4, self.seq_length
            )
        elif positional_encoding_type == "fixed":
            self.position_encoding = FixedPositionalEncoding(
                self.embedding_dim,
            )
            self.position_encoding_4 = FixedPositionalEncoding(
                self.embedding_dim_4,
            )

        self.pe_dropout = nn.Dropout(p=self.dropout_rate)

        self.transformer = TransformerModel(
            embedding_dim,
            num_layers,
            num_heads,
            hidden_dim,

            self.dropout_rate,
            self.attn_dropout_rate,
        )

        self.transformer_4 = TransformerModel(
            embedding_dim_4,
            num_layers,
            num_heads,
            hidden_dim,

            self.dropout_rate,
            self.attn_dropout_rate,
        )

        self.pre_head_ln = nn.LayerNorm(embedding_dim)
        self.pre_head_ln_4 = nn.LayerNorm(embedding_dim_4)

        if self.conv_patch_representation:

            self.conv_x = nn.Conv3d(
                256,
                self.embedding_dim,
                kernel_size=3,
                stride=1,
                padding=1
            )

        if self.conv_patch_representation:

            self.conv_x_4 = nn.Conv3d(
                128,
                self.embedding_dim,
                kernel_size=3,
                stride=1,
                padding=1
            )

        #Encoder
        self.Unet = Base3DUNet(inplanes = 16, num_classes = 4 , width = 16)
        self.bn = nn.BatchNorm3d(256)
        self.bn_4 = nn.BatchNorm3d(128)
        self.relu = nn.ReLU(inplace=True)


    def encode(self, x):
        if self.conv_patch_representation:
            # combine embedding with conv patch distribution
            x1_1, x2_1, x3_1, x4_1, x = self.Unet(x)

            x4_t = self.bn_4(x4_1) #
            x4_t= self.relu(x4_t) #
            x4_t= self.conv_x_4(x4_t) #
            x4_t = x4_t.permute(0, 2, 3, 4, 1).contiguous() #
            x4_t= x4_t.view(x4_t.size(0), -1, self.embedding_dim_4) #

            x = self.bn(x)
            x = self.relu(x)
            x = self.conv_x(x)
            x = x.permute(0, 2, 3, 4, 1).contiguous()
            x = x.view(x.size(0), -1, self.embedding_dim)

        else:
            x = self.Unet(x)
            x = self.bn(x)
            x = self.relu(x)
            x = (
                x.unfold(2, 2, 2)
                .unfold(3, 2, 2)
                .unfold(4, 2, 2)
                .contiguous()
            )
            x = x.view(x.size(0), x.size(1), -1, 8)
            x = x.permute(0, 2, 3, 1).contiguous()
            x = x.view(x.size(0), -1, self.flatten_dim)
            x = self.linear_encoding(x)

        x4_t = self.position_encoding_4(x4_t) #
        x4_t = self.pe_dropout(x4_t) #

        x = self.position_encoding(x)
        x = self.pe_dropout(x)

        # apply transformer
        x4_t, intmd_x4_t = self.transformer_4(x4_t) #
        x4_t = self.pre_head_ln_4(x4_t) #

        x, intmd_x = self.transformer(x)
        x = self.pre_head_ln(x)

        return x1_1, x2_1, x3_1, x4_1, x, intmd_x, x4_t, intmd_x4_t

    def decode(self, x):
        raise NotImplementedError("Should be implemented in child class!!")

    def forward(self, x, auxillary_output_layers=[1, 2, 3, 4], auxillary_output_layers_4=[1, 2, 3, 4]):

        x1_1, x2_1, x3_1, x4_1, encoder_output, intmd_encoder_outputs, encoder_output_4, intmd_encoder_outputs_4 = self.encode(x)

        decoder_output = self.decode(
            x1_1, x2_1, x3_1, x4_1, encoder_output, intmd_encoder_outputs, encoder_output_4, intmd_encoder_outputs_4, auxillary_output_layers, auxillary_output_layers_4
        )

        if auxillary_output_layers is not None:
            auxillary_outputs = {}
            for i in auxillary_output_layers:
                val = str(2 * i - 1)
                _key = 'Z' + str(i)
                auxillary_outputs[_key] = intmd_encoder_outputs[val]

        if auxillary_output_layers_4 is not None:
            auxillary_outputs_4 = {}
            for i in auxillary_output_layers_4:
                val_4 = str(2 * i - 1)
                _key_4 = 'Z' + str(i)
                auxillary_outputs_4[_key_4] = intmd_encoder_outputs_4[val_4]


        return decoder_output

    def _get_padding(self, padding_type, kernel_size):
        assert padding_type in ['SAME', 'VALID']
        if padding_type == 'SAME':
            _list = [(k - 1) // 2 for k in kernel_size]
            return tuple(_list)
        return tuple(0 for _ in kernel_size)

    def _reshape_output(self, x):
        x = x.view(
            x.size(0),
            int(self.img_dim / self.patch_dim),
            int(self.img_dim / self.patch_dim),
            int(self.img_dim / self.patch_dim),
            self.embedding_dim,
        )
        x = x.permute(0, 4, 1, 2, 3).contiguous()

        return x

    def _reshape_output_4(self, x):
        x = x.view(
            x.size(0),
            int(self.img_dim*2 / self.patch_dim),
            int(self.img_dim*2 / self.patch_dim),
            int(self.img_dim *2/ self.patch_dim),
            self.embedding_dim,
        )
        x = x.permute(0, 4, 1, 2, 3).contiguous()

        return x


class AttentionBlock(nn.Module):
    # Xs come from the encoder and Gs come from previous lower layer and a point-wise conv is applied to both and
    # their sum will be calculated and fed into ReLU Then another point-wise conv (Psi) is applied with a sigmoid after
    # that, which is supposed to be the probability map of each data point (pixel) hence it is multiplied with the X

    # Why we don't multiply it with G?
    # My Intuition: because more information will be available in X and its size is closer to the current module
    # if G is used then we need rescaling or padding, which makes it useless, on the other hand we already have the
    # effect of both
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(nn.Conv3d(F_g, F_int, kernel_size=1, stride=1, padding=0), nn.BatchNorm3d(F_int))
        self.W_x = nn.Sequential(nn.Conv3d(F_l, F_int, kernel_size=1, stride=1, padding=0), nn.BatchNorm3d(F_int))
        self.psi = nn.Sequential(nn.Conv3d(F_int, 1, kernel_size=1, stride=1, padding=0), nn.BatchNorm3d(1))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g = self.W_g(g)
        x_o = self.W_x(x)
        relu_sum = self.relu(g + x_o)
        psi = self.psi(relu_sum)
        psi = torch.sigmoid(psi)
        return x * psi


class Attention3UNet(Base3DUNet):  # Generalization of 2D attention UNet
    def __init__(self, in_channels=3, out_channels=3, features=[64, 128, 256, 512], up_sample=True):
        super().__init__(in_channels, out_channels, features, up_sample=up_sample)
        self.attention = nn.ModuleList()

        for feature in reversed(features):
            if up_sample:
                self.attention.append(AttentionBlock(feature * 2, feature, feature))
            else:
                self.attention.append(AttentionBlock(feature, feature, feature))
                # when convT is used, then the G is normal and the same as skip

    def forward(self, inputs):
        skips = []

        x = inputs
        for down in self.downs:
            x = down(x)
            skips.append(x)
            x = self.pooling(x)

        x = self.bottleneck(x)

        # same as original Unet till here

        for idx in range(0, len(self.ups), 2):  # going up 2 steps, as each step has convTranspose and DoubleConv
            x = self.ups[idx](x)  # up sampling w/ the convTranspose or Upsampler

            # ---> DIFF START: the skip_connection acquired is the X_l in attention UNet paper, and x is G
            skip_connection = skips.pop()  # give me the last skip I added, to add it first on the ups
            skip_connection = self.attention[idx // 2](x, skip_connection)  # (G,X) = (x, skip_connection) in here
            # <---- end of difference between attention and base 3D UNet

            x = torch.cat((skip_connection, x), dim=1)  # dim 0 is batch, dim 1 is the channels
            x = self.ups[idx + 1](x)  # double conv

        return self.output_conv(x)



In [ ]:
# DSC = 2 * |A intersect B| / (|A| + |B|)
class DiceLoss(nn.Module):
    """Calculate dice loss."""

    def __init__(self, eps: float = 1e-9):
        super(DiceLoss, self).__init__()
        self.eps = eps

    def forward(self,
                logits: torch.Tensor,
                targets: torch.Tensor) -> torch.Tensor:
        num = targets.size(0)
        probability = torch.sigmoid(logits)
        probability = probability.view(num, -1)
        targets = targets.view(num, -1)
        assert (probability.shape == targets.shape)

        intersection = 2.0 * (probability * targets).sum()
        union = probability.sum() + targets.sum()
        dice_score = (intersection + self.eps) / union
        # print("intersection", intersection, union, dice_score)
        return 1.0 - dice_score


class BCEDiceLoss(nn.Module):
    """Compute objective loss: BCE loss + DICE loss."""

    def __init__(self):
        super(BCEDiceLoss, self).__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = DiceLoss()

    def forward(self,
                logits: torch.Tensor,
                targets: torch.Tensor) -> torch.Tensor:
        assert (logits.shape == targets.shape)
        dice_loss = self.dice(logits, targets)
        bce_loss = self.bce(logits, targets.float())

        return bce_loss + dice_loss

In [ ]:
BATCH_SIZE = 4
EPOCHS = 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
LR = 0.0001

In [ ]:
train_dl = get_dl(get_train_ds(), BATCH_SIZE,nw=2)
val_dl = get_dl(get_val_ds(), BATCH_SIZE, nw=2)

images: 221, masks: 221 
images: 148, masks: 148 


In [ ]:
model = Attention3UNet(3, 3, features=[64, 128, 256, 512], up_sample=False).to(DEVICE)
print(model)

TypeError: ignored

In [ ]:
print(f"total parameters = {sum(p.numel() for p in model.parameters())}") # 5m more params than base3DUnet
print(f"total learnable parameters = {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

total parameters = 91005199
total learnable parameters = 91005199


In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=LR)
# loss = torch.nn.BCEWithLogitsLoss # cannot be used because there's a lot of imbalance anyway, so it is better to combine it with dice
loss = BCEDiceLoss()
# loss = DiceLoss()

In [ ]:
train(model, epochs=EPOCHS, training_loader=train_dl, loss_fn=loss, device=DEVICE, optimizer=opt)

At epoch [19/100]:  20%|█▉        | 11/56 [00:23<01:22,  1.83s/it, loss=0.596]